In [1]:
import pandas as pd
import altair as alt
import numpy as np

In [2]:
def extrae_drive(url):
    edit = '/edit#gid='
    export ='/export?format=xlsx&gid='
    url = (url).replace(edit, export)
    return url

## Generando la Base principal
* Se usara la base de **cesados** y la base **Planilla**, esta sera filtrada segun las condiciones de la conclusion
* En esta nueva base se empezara a agregar el resto de informacion para el proyecto

In [3]:
cesados = extrae_drive('https://docs.google.com/spreadsheets/d/1bO_T0lAQgjcpjFFNVAwiXd2obwRjJS3K/edit#gid=2050623625')
cesados = pd.read_excel(
                cesados,
                usecols='E,I,O,P,S,T,W,X'
            ).rename(
                columns = {
                    'Total o Temprana':'TIEMPO_CESE',
                    'Producto':'PRODUCTO',
                    'TIPO':'TIPO_CESE',
                    'MOTIVO PRIMARIO':'MOTIVO_CESE'}
            )

planilla = extrae_drive('https://docs.google.com/spreadsheets/d/1L094YBwj3BvobbXb4t3ahzpmL6gBSxgs/edit#gid=648976010')
planilla = pd.read_excel(
                planilla,
                usecols='A,B,D:H'
            ).rename(
                columns={
                    'COD':'CODIGO',
                    'REGION/DPTO':'REGION',
                    'DIVISION/GC':'DIVISION'    }
            )

* Al aprecer hay colaboradores que cesaron mas de dos veces, es decir que ingresaron, cesaron, volvieron a ingresar y volvieron a cesar
* Se tomara en cuenta este numero de ceses como una caracteristca (aunque son pocas las personas) ***Revisar**
* Eliminare los colaboradores duplicados considerando solo los ultimos ceses, pues son de los que se tendria data mas reciente
* Genero la columna antiguedad definida como CESE menos INGRESO en meses

In [4]:
# Colaboradores con mas de 2 ceses
cesados.groupby(by = 'CODIGO', as_index=False).agg(n = ('CODIGO','count')).query('n>=2')

,CODIGO,n
425,A22AAAAA34,2
535,A33AAAA149,2
580,A33AAAA21A,2
642,A34AAAAA55,2
688,A36AAAAA18,2
1151,A4AAAAA57A,2
1212,A4AAAAA742,2
1397,A57AAAAA17,2
1408,A57AAAAA37,2
1938,A84AAAAA56,2


In [5]:
cesados = cesados.sort_values(
    by = 'CESE',
    ascending = False # de mayor a menor
).drop_duplicates(
    'CODIGO',
    keep = 'first' # los mas recientes
).iloc[:, [0,2,3,6,7]] #solo selecciono las columnas que usare

In [6]:
cesados['antiguedad_meses'] = (((cesados['CESE'] - cesados['INGRESO']).dt.days)/30).round(0) # Creo antiguedad

* Utilizo como base Planilla que tiene un registro de colabores cesados y activos
* Elimino los duplicados pues, la planilla cuenta con registro de colaboradores por meses, me quedo solo con los mas recientes
* Realizo los filtros para generar la base que quiero

In [7]:
planilla = planilla.drop_duplicates(
    'CODIGO',
    keep = 'last').iloc[:, 1:6] # no me interesa la fecha, solo los valores unicos del codigo

In [71]:
df = pd.merge(
    planilla,
    cesados,
    on = 'CODIGO',
    how = 'left'
).query('PRODUCTO == "GRUPAL"')

* Genero una base para Grupal y Otra para invidual porque seran dos diferentes proyectos con caracteristicas diferentes
* Empiezo a filtrar, Cargo, Tipo de cese, motivo y antiguedad
    * Como uni la base de ceses con planilla se han generado vacios de los colaboradores activos, para ellos debo de tener cuidado en los filtros

In [72]:
df = df[
    (df['CARGO'].isin(['VENTA GRUPAL']))
    & (df['TIPO_CESE'] != 'INVOLUNTARIA') #vacio o voluntaria
    & (~df['MOTIVO_CESE'].isin(['Personal Eventual', 'No le Gusto el Trabajo'])) # los otros motivos o vacios
    & (df['antiguedad_meses'] >= 9) # quito los mayores a 9 meses
    | (df['antiguedad_meses'].isna())] # o que sean activos

* Creo una nueva columna que clasifique si es activo o cesado, usando el TIPO_CESE como referencia
* Reemplazo la fecha de los que no cesaron con la fecha de cierre, es decir el 01/01/2022
* Selecciono solo las variables que usare:
    * Ya no usare INGRESO porque necesito extraerno de otra base donde tenga el ingreso de los activos y cesados
    * Ya no usare TIPO_CESE,  MOTIVO_CESE ya no competen en el analisis
    * Ya no usare antiguedad_meses, pues generare una nueva

In [73]:
df.loc[df['TIPO_CESE'] == 'VOLUNTARIA', 'Estado'] = 'CESADO'
df.loc[df['TIPO_CESE'] != 'VOLUNTARIA', 'Estado'] = 'ACTIVO'
df.loc[df['CESE'].isna(), 'CESE'] = '2022-01-01'
df = df.iloc[:,[0,1,2,3,4,6,10]]

* Esta es la base que usare como principal donde ire agregando la informacion de otras variables

In [76]:
df.sample(5)

,CODIGO,CARGO,AGENCIA,PRODUCTO,REGION,CESE,Estado
5056,111AAAAA24,VENTA GRUPAL,OFI HUANUCO,GRUPAL,TERR CENTRO,2022-01-01,ACTIVO
846,AA8AAA3277,VENTA GRUPAL,OFI COMAS,GRUPAL,TERR LIMA NORTE,2019-05-31,CESADO
173,AA8AAA2877,VENTA GRUPAL,OFI CANTO GRANDE SJL,GRUPAL,TERR SAN JUAN,2018-05-20,CESADO
3359,A4AAAAA283,GACG,OFI SULLANA,GRUPAL,TERR NORTE 2,2022-01-01,ACTIVO
5137,A39AAAAA31,VENTA GRUPAL,OFI TRUJILLO NORTE,GRUPAL,TERR LA LIBERTAD,2022-01-01,ACTIVO


In [77]:
df.isna().sum() #No tengo NAS hasta el mento

CODIGO      0
CARGO       0
AGENCIA     0
PRODUCTO    0
REGION      0
CESE        0
Estado      0
dtype: int64

* Es necesario ver si la base con la que vamos a continuar esta balanceada
    *  Como se muestra tenemos **30% de Cesados y 70% de Activos**, por lo que podemos continuar con el analisis

In [97]:
df.groupby(
    by = 'Estado',
    as_index = False
).agg(
    n = ('CODIGO' , 'count')
)['n'] / df.shape[0]

0    0.693061
1    0.306939
Name: n, dtype: float64

## Agregando variables a la base principal

### Bonos de Venta Grupal
* Este valor diferencia el ingreso que tiene un colaborador de otro, ya que tienen un sueldo base y comisiones
* Dentro del calculo del bono existen varios tipos, en este caso se toma el tipo normal que es el estandar de calculo
*  

In [104]:
bonos_venta = extrae_drive('https://docs.google.com/spreadsheets/d/1POTE8j5SneZ0QSvmbtKL_t4hu2odWlTM/edit#gid=578896078')
bonos_venta = pd.read_excel(
                    bonos_venta,
                    usecols = 'A, B, CE'
                ).rename(
                    columns = {
                    'Codigo Trabajador': 'CODIGO',
                    'RV TOTAL': 'RV'}
                )

In [106]:
bonos_venta.sample(5)

,MES,CODIGO,RV
34175,2020-03-01,A53AAAAA27,717.288
61316,2021-09-01,1A3AAAAA25,2681.965
45770,2020-10-01,A4AAAAA767,9479.600
9191,2018-09-01,AA7AAA4338,480.000
9656,2018-09-01,A55AAAAAA1,4511.700




programa_mejora = extrae_drive('https://docs.google.com/spreadsheets/d/1CKFlLpx248yvZS0INAawAA5e8NC2qdvV/edit#gid=1510069928')
memos = extrae_drive('https://docs.google.com/spreadsheets/d/1Q2pvICQWLAh8Rvou9FvH085KULXioDVK/edit#gid=238204385')
vacaciones = extrae_drive('https://docs.google.com/spreadsheets/d/1riJOPN5KUwV1962wO6mjPL7PMgL2v3_s/edit#gid=800276956')
descanso_medico = extrae_drive('https://docs.google.com/spreadsheets/d/1H_oENDjS0FyDNVmSg1199PrE6s8q6z7-/edit#gid=1138295584')
bonos_venta_i = extrae_drive('https://docs.google.com/spreadsheets/d/1Vjh5S86c1r60L-CnuQB_m6JZPy223Z4p/edit#gid=100516943834575165411')

desempeno = extrae_drive()
reconocimientos = extrae_drive()
hijxs = extrae_drive()
datos_personales = extrae_drive()
recategorizaciones = extrae_drive()
cobertura_ofi = extrae_drive()
antg_oficina = extrae_drive()
gptw_oficina = extrae_drive()